In [1]:
import sys, os
sys.path.append("/home/daniil/repos/pytorch-segmentation-detection/")
sys.path.insert(0, '/home/daniil/repos/pytorch-segmentation-detection/vision/')

os.environ["CUDA_VISIBLE_DEVICES"] = '2'

import torch
import pytorch_segmentation_detection.models.resnet_dilated as resnet_dilated
from pytorch_segmentation_detection.utils.flops_benchmark import add_flops_counting_methods

fcn = resnet_dilated.Resnet18_32s(num_classes=21)

/home/daniil/repos/anaconda2/lib/python2.7/site-packages/torch/nn/modules/module.py:482: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  own_state[name].copy_(param)


In [2]:
from conditional_computation import Resnet18_8s_gated, GatedResidualUnit, GatedResidualUnitSpatialSimple

from PIL import Image
from matplotlib import pyplot as plt

import torch
from torchvision import transforms
import torchvision
from torch.autograd import Variable
import pytorch_segmentation_detection.models.resnet_dilated as resnet_dilated
from pytorch_segmentation_detection.layers import GlobalAvgPool2d

import numpy as np
import torch.nn as nn

from pytorch_segmentation_detection.datasets.pascal_voc import PascalVOCSegmentation

import pytorch_segmentation_detection.models.fcn as fcns
import pytorch_segmentation_detection.models.resnet_dilated as resnet_dilated
from pytorch_segmentation_detection.transforms import (ComposeJoint,
                                                       RandomHorizontalFlipJoint,
                                                       RandomScaleJoint,
                                                       CropOrPad,
                                                       ResizeAspectRatioPreserve,
                                                       RandomCropJoint,
                                                       Split2D)


train_transform = ComposeJoint(
                [
                    RandomHorizontalFlipJoint(),
                    RandomCropJoint(crop_size=(224, 224)),
                    [transforms.ToTensor(), None],
                    [transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), None],
                    [None, transforms.Lambda(lambda x: torch.from_numpy(np.asarray(x)).long()) ]
                ])


trainset = PascalVOCSegmentation('/home/daniil/projects/pascal/dataset/',
                                 download=False,
                                 joint_transform=train_transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=10,
                                          shuffle=True, num_workers=4, drop_last=True)


In [3]:
fcn = add_flops_counting_methods(fcn)
fcn = fcn.cuda().train()

fcn.start_flops_count()

loader = iter( trainloader )

data = loader.next()

batch = Variable(data[0].cuda())

_ = fcn(batch) # works now

/home/daniil/repos/anaconda2/lib/python2.7/site-packages/torch/nn/functional.py:1423: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.")


In [4]:
fcn.compute_average_flops_cost() / 1e9 / 2 # Result in GFLOPs

# All models with 32 output stride should be more-or-less consistent with this page:
# https://github.com/albanie/convnet-burden
# Which is the case for all resnet models that we have checked.

1.8140887065